<a href="https://colab.research.google.com/github/NikweT/tech-promachinelearning/blob/main/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing panda

In [ ]:
import pandas as pd


loading all the tables which are in the csv format

In [ ]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
tags = pd.read_csv('tags.csv')
links = pd.read_csv('links.csv')

merge the ratings table with movies table

In [ ]:
merged = pd.merge(ratings, movies, on='movieId', how='left')

merge the tags table with the ratings table

In [ ]:
merged = pd.merge(merged, tags, on=['userId', 'movieId'], how='left')

merging the links table with the ratings table

In [ ]:
merged = pd.merge(merged, links, on='movieId', how='left')

see the merged dataset

In [ ]:
merged.head()

,userId,movieId,rating,timestamp_x,title,genres,tag,timestamp_y,imdbId,tmdbId
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN,114709,862.0
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,NaN,NaN,113228,15602.0
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,NaN,NaN,113277,949.0
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,NaN,NaN,114369,807.0
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,NaN,NaN,114814,629.0


import libraries and perform Logistic regression result

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [38]:
merged['high_rating'] = (merged['rating'] >= 4).astype(int)
y = merged['high_rating']

In [39]:
from sklearn.preprocessing import OneHotEncoder

X = merged[['movieId', 'genres']]
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X[['genres']])

X_final = pd.concat([
    pd.DataFrame(X['movieId'].values, columns=['movieId']),
    pd.DataFrame(X_encoded.toarray(), columns=encoder.get_feature_names_out(['genres']))
], axis=1)


In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)


In [49]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [48]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.60      0.69      0.64     10525
           1       0.61      0.51      0.55     10011

    accuracy                           0.60     20536
   macro avg       0.60      0.60      0.60     20536
weighted avg       0.60      0.60      0.60     20536



In [46]:

X_final = pd.DataFrame(
    X_encoded.toarray(),
    columns=encoder.get_feature_names_out(['genres'])
)


In [47]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000, solver='liblinear')
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000, solver='liblinear')

interpret the genre impact

In [51]:
coefficients = pd.DataFrame({
    'Genre': encoder.get_feature_names_out(['genres']),
    'Coefficient': model.coef_[0]
}).sort_values(by='Coefficient', ascending=False)

print(coefficients)


                                                 Genre  Coefficient
796                          genres_Crime|Thriller|War     1.877638
103     genres_Action|Adventure|Drama|Thriller|Western     1.876953
908                  genres_Film-Noir|Romance|Thriller     1.799972
648                 genres_Comedy|Crime|Drama|Thriller     1.781466
781                    genres_Crime|Film-Noir|Thriller     1.696609
..                                                 ...          ...
53       genres_Action|Adventure|Comedy|Crime|Thriller    -1.852464
36   genres_Action|Adventure|Children|Comedy|Fantas...    -1.883089
303                       genres_Action|Fantasy|Sci-Fi    -1.925802
223                genres_Action|Comedy|Sci-Fi|Western    -2.050779
115           genres_Action|Adventure|Fantasy|Thriller    -2.490950

[951 rows x 2 columns]


Random Forest Code

In [61]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd

X = merged[['genres']]
y = (merged['rating'] >= 4).astype(int)

encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X[['genres']])

X_final = pd.DataFrame(
    X_encoded.toarray(),
    columns=encoder.get_feature_names_out(['genres'])
)

X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.2, random_state=42
)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
print(classification_report(y_test, y_pred_rf))

feature_importance = pd.DataFrame({
    'Genre': encoder.get_feature_names_out(['genres']),
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print(feature_importance.head(10))


KeyboardInterrupt: 